# External delta circuit

This example shows how to create an external delta circuit and connect it with a Maxwell 2D design.

Keywords: **Maxwell 2D**, **Circuit**, **netlist**

## Perform imports and define constants

In [1]:
import os
import tempfile
import time

import ansys.aedt.core

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Launch AEDT and Maxwell 2D

Launch AEDT and Maxwell 2D providing the version, path to the project, and the graphical mode.

In [4]:
project_name = os.path.join(temp_folder.name, "Maxwell_circuit_example.aedt")
design_name = "1 Maxwell"
circuit_name = "2 Delta circuit"

In [5]:
m2d = ansys.aedt.core.Maxwell2d(
    version=AEDT_VERSION,
    new_desktop=False,
    design=design_name,
    project=project_name,
    solution_type="TransientXY",
    non_graphical=NG_MODE,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_43a8490a-6299-44bd-a73a-bcaae789e682.log is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project Maxwell_circuit_example has been created.


PyAEDT INFO: Added design '1 Maxwell' of type Maxwell 2D.


PyAEDT INFO: Aedt Objects correctly read


## Initialize dictionaries and define variables

Initialize dictionaries that contain all design variable definitions.

In [6]:
voltage = "230V"
frequency = "50Hz"

In [7]:
transient_parameters = {
    "voltage": voltage,
    "frequency": frequency,
    "electric_period": "1 / frequency s",
    "stop_time": "2 * electric_period s",
    "time_step": "electric_period / 20 s",
}

In [8]:
circuit_parameters = {
    "voltage": voltage,
    "frequency": frequency,
    "resistance_value": "1Ohm",
}

In [9]:
for k, v in transient_parameters.items():
    m2d[k] = v

## Create geometry

Create copper coils and a vacuum region. Assign mesh operations, and then assign a balloon boundary to the region edges.

In [10]:
coil1_id = m2d.modeler.create_circle(
    orientation="Z", origin=[0, 0, 0], radius=10, name="coil1", material="copper"
)
coil2_id = m2d.modeler.create_circle(
    orientation="Z", origin=[25, 0, 0], radius=10, name="coil2", material="copper"
)
coil3_id = m2d.modeler.create_circle(
    orientation="Z", origin=[50, 0, 0], radius=10, name="coil3", material="copper"
)

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


In [11]:
region = m2d.modeler.create_region(pad_value=[200, 400, 200, 400])

In [12]:
m2d.mesh.assign_length_mesh(
    assignment=[coil1_id, coil2_id, coil3_id, region], maximum_length=5
)

PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


length_XCQK1F

In [13]:
m2d.assign_vector_potential(assignment=region.edges, vector_value=0)

PyAEDT INFO: Boundary Vector Potential Vector_KOV85F has been created.


Vector_KOV85F

## Assign excitations

Assign external windings

In [14]:
wdg1 = m2d.assign_winding(
    assignment=["coil1"],
    is_solid=False,
    winding_type="External",
    name="winding1",
    current=0,
)
wdg2 = m2d.assign_winding(
    assignment=["coil2"],
    is_solid=False,
    winding_type="External",
    name="winding2",
    current=0,
)
wdg3 = m2d.assign_winding(
    assignment=["coil3"],
    is_solid=False,
    winding_type="External",
    name="winding3",
    current=0,
)

PyAEDT INFO: Boundary Winding winding1 has been created.


PyAEDT INFO: Boundary Coil Coil_S2CCTP has been created.


PyAEDT INFO: Boundary Winding winding2 has been created.


PyAEDT INFO: Boundary Coil Coil_UNUJS9 has been created.


PyAEDT INFO: Boundary Winding winding3 has been created.


PyAEDT INFO: Boundary Coil Coil_OP4EGH has been created.


## Create simulation setup

Create the simulation setup, defining the stop time and time step.

In [15]:
setup = m2d.create_setup()
setup["StopTime"] = "stop_time"
setup["TimeStep"] = "time_step"

## Create external circuit

Create the circuit design, including all windings of type ``External`` in the Maxwell design.

In [16]:
circuit = m2d.create_external_circuit(circuit_design=circuit_name)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 2636!


PyAEDT INFO: No project is defined. Project Maxwell_circuit_example exists and has been read.


PyAEDT INFO: Added design '2 Delta circuit' of type Maxwell Circuit.


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerMaxwellCircuit class has been initialized!


## Define variables from dictionaries

Define design variables from the created dictionaries.

In [17]:
for k, v in circuit_parameters.items():
    circuit[k] = v

In [18]:
windings = [
    circuit.modeler.schematic.components[k]
    for k in list(circuit.modeler.schematic.components.keys())
    if circuit.modeler.schematic.components[k].parameters["Info"] == "Winding"
]

## Finalize external circuit

Draw these other components: resistors, voltage sources, and grounds

In [19]:
circuit.modeler.schematic_units = "mil"

In [20]:
resistors = []
v_sources = []
ground = []

In [21]:
for i in range(len(windings)):
    r = circuit.modeler.schematic.create_resistor(
        name="R" + str(i + 1), value="resistance_value", location=[1000, i * 1000]
    )
    resistors.append(r)

    v = circuit.modeler.schematic.create_component(
        component_library="Sources",
        component_name="VSin",
        location=[2000, i * 1000],
        angle=90,
    )
    v_sources.append(v)

    v.parameters["Va"] = "voltage"
    v.parameters["VFreq"] = "frequency"
    v.parameters["Phase"] = str(i * 120) + "deg"

    g = circuit.modeler.schematic.create_gnd([2300, i * 1000], angle=90)
    ground.append(g)

## Connect components

Connect the components by drawing wires.

In [22]:
windings[2].pins[1].connect_to_component(resistors[2].pins[0], use_wire=True)
windings[1].pins[1].connect_to_component(resistors[1].pins[0], use_wire=True)
windings[0].pins[1].connect_to_component(resistors[0].pins[0], use_wire=True)

True

In [23]:
resistors[2].pins[1].connect_to_component(v_sources[2].pins[1], use_wire=True)
resistors[1].pins[1].connect_to_component(v_sources[1].pins[1], use_wire=True)
resistors[0].pins[1].connect_to_component(v_sources[0].pins[1], use_wire=True)

True

In [24]:
circuit.modeler.schematic.create_wire(
    points=[
        resistors[2].pins[1].location,
        [1200, 2500],
        [-600, 2500],
        [-600, 0],
        windings[0].pins[0].location,
    ]
)
circuit.modeler.schematic.create_wire(
    points=[
        resistors[1].pins[1].location,
        [1200, 1500],
        [-300, 1500],
        [-300, 2000],
        windings[2].pins[0].location,
    ]
)
circuit.modeler.schematic.create_wire(
    points=[
        resistors[0].pins[1].location,
        [1200, 500],
        [-300, 500],
        [-300, 1000],
        windings[1].pins[0].location,
    ]
)

Class: ansys.aedt.core.modeler.circuits.object_3d_circuit.Wire

## Export and import netlist

Export the netlist file and then import it to Maxwell 2D.

In [25]:
netlist_file = os.path.join(temp_folder.name, "_netlist.sph")
circuit.export_netlist_from_schematic(netlist_file)

'C:\\Users\\ansys\\AppData\\Local\\Temp\\tmpt1h9_r4x.ansys\\_netlist.sph'

In [26]:
m2d.edit_external_circuit(
    netlist_file_path=netlist_file, schematic_design_name=circuit_name
)

PyAEDT INFO: Active Design set to 1;2 Delta circuit


True

## Analyze setup


In [27]:
setup.analyze(cores=NUM_CORES)

PyAEDT INFO: Project Maxwell_circuit_example Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving design setup MySetupAuto


PyAEDT INFO: Design setup MySetupAuto solved correctly in 0.0h 0.0m 30.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


## Create rectangular plot

Plot winding currents

In [28]:
m2d.post.create_report(
    expressions=["Current(winding1)", "Current(winding2)", "Current(winding3)"],
    domain="Sweep",
    primary_sweep_variable="Time",
    plot_name="Winding Currents",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpt1h9_r4x.ansys\Maxwell_circuit_example.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpt1h9_r4x.ansys\Maxwell_circuit_example.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.0442967414855957


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: No report category provided. Automatically identified Transient


Winding Currents

## Release AEDT

In [29]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project Maxwell_circuit_example Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_dir.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [30]:
temp_folder.cleanup()